In [1]:
from robotic import ry
import numpy as np
import time

###########################################PART-1 STARTS HERE##########################################

In [2]:
C = ry.Config()
C.addFile('cargobot_base.g')
C.addFile('maze.g')

qHome = C.getJointState()
C.setJointState(qHome)

Get Final Pose with KOMO

In [3]:
komo = ry.KOMO(C, 1, 1, 2, True)
komo.addControlObjective([], 0, 1e-1)
komo.addControlObjective([], 2, 1e0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e2], [], order=1)


ret = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
print(ret)
q = komo.getPath()
print('size of path:', q.shape)

qT = komo.getPath()[0]
C.setJointState(qT)
C.view(False, "IK solution")


{ time: 0.000907, evals: 13, done: 1, feasible: 0, sos: 7.38487, f: 0, ineq: 0, eq: 755 }
size of path: (1, 3)


0

RRT

In [7]:
#define a path finding problem
rrt = ry.PathFinder()
rrt.setProblem(C, [qHome], [qT])

for i in range(0, 2):
    ret = rrt.solve()
    print("rettttt: ", ret)
    path = ret.x
    print("path size:", path.shape[0])
    # display the path
    for t in range(0, path.shape[0] - 1):
        C.setJointState(path[t])
        C.view()
        time.sleep(0.002)

rettttt:  { time: 0.280243, evals: 4166, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
path size: 584
rettttt:  { time: 0.281683, evals: 4183, done: 1, feasible: 1, sos: -1, f: -1, ineq: -1, eq: -1 }
path size: 584


Trajectory Optimization via KOMO

In [10]:
C.setJointState(qHome)
komo2 = ry.KOMO(C, 1, path.shape[0], 2, True)
komo2.addControlObjective([], 0, 1e-1)
komo2.addControlObjective([], 2, 1e0)
komo2.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.positionDiff, ['base', 'goal_area'], ry.OT.eq, [1e2])
komo2.addObjective([1], ry.FS.jointState, [], ry.OT.eq, [1e1], [], order=1)
komo2.initWithPath_qOrg(path)

ret = ry.NLP_Solver(komo2.nlp(), verbose=0).solve()
q = komo2.getPath()
print('last path:', q)

# display the path
for i in range(0, q.shape[0]):
    C.setJointState(q[i])
    C.view(False, f'waypoints{i}')
    time.sleep(0.02)

C.view()

last path: [[ 3.55947085e-05 -1.02135294e-03  7.10072083e-05]
 [ 1.11429655e-04 -3.04723468e-03  2.12910069e-04]
 [ 2.32150369e-04 -6.06082107e-03  4.25597025e-04]
 ...
 [-4.49895567e+00 -2.99998152e+00  3.17860040e+00]
 [-4.50001143e+00 -2.99999780e+00  3.17863077e+00]
 [-4.50001139e+00 -2.99999781e+00  3.17863076e+00]]


0

###########################################PART-1 ENDS HERE##########################################

###########################################PART-2 STARTS HERE##########################################

In [ ]:
del komo
del C

###########################################PART-2 ENDS HERE##########################################